# Esercizio 5 - Soluzione

### Parametri in input

In [1]:
gtf_file_name = './input.gtf'
reference_file_name = './ENm006.fa'

Scegliere una feature tra `exon` per ricostruire i trascritti e `CDS` per ricostruire le *coding sequences* del gene.

In [2]:
#selected_feature = 'exon'
selected_feature = 'CDS'

### 1) Importazione del modulo `re`

In [3]:
import re

### 2) Definizione di una funzione per estrarre da un *record* GTF il gene e il trascritto associato

La funzione prende come argomento un *record* GTF e restituisce la tupla `(gene_id, transcript_id)` contenente il gene e il trascritto associati.

In [4]:
def get_gene_and_transcript(gtf_record):
    
    gene_id = re.search(r'gene_id\s+"(.+?)";', gtf_record).group(1)
    
    transcript_id = re.search(r'transcript_id\s+"(.+?)";', gtf_record).group(1)
    
    return (gene_id, transcript_id)

### 3) Definizione di una funzione per formattare in `FASTA` una sequenza

La funzione prende come argomenti un *header* `FASTA` (che inizia con un simbolo `>`), una sequenza nucleotidica (o di proteina) e la dimensione (passata come *keyword* `record` con valore di *default* pari a 80) dei *record* in cui separare la sequenza. La funzione restituisce una stringa contenente la sequenza in formato `FASTA`.

*Suggerimento*: usare la funzione `compile()` per creare un'espressione regolare.

In [5]:
def format_fasta(header, sequence, record = 80):    
    p = re.compile('\w{,' + str(record) + '}')
    return header + '\n' + '\n'.join(re.findall(p, sequence))

### 4) Definizione di una funzione per eseguire il *reverse and complement* di una sequenza nucleotidica

La funzione prende come argomento una sequenza nucleotidica e restituisce il *reverse and complement* della sequenza in lettere maiuscole.

In [6]:
def reverse_complement(sequence):
    complement_dict = {'A' : 'T', 'C' : 'G', 'G' : 'C', 'T' : 'A'}    
    return ''.join(complement_dict[base] for base in sequence[::-1].upper())

In [7]:
sequence = 'ATGgGAaACc'
reverse_complement(sequence)

'GGTTTCCCAT'

### 5) Definizione di una funzione che ricostruisce la sequenza di un trascritto/coding sequence

La funzione prende i seguenti argomenti:
- una lista di tuple `(start, end, ...)` che rappresentano le *features* `exon` che compongono un trascritto oppure le *features* `CDS` che compongono una *coding sequence*
- la sequenza della genomica di riferimento
- un valore di *strand* (`+/-`)

e ricostruisce la sequenza (del trascritto/CDS) tenendo conto dello strand specificato.

**NB**: se lo strand è `+` la *feature* di coordinate minori è la prima *feature* che compone la sequenza da ricostruire, altrimenti è l'ultima.

**Esempio1**: *features* `exon` che compongono la sequenza del trascritto `XX-FW83563B9.4-002` del gene `ATP6AP1`:

    ENm006	VEGA_Known	exon	542747	542902		.	+	. 
    ENm006	VEGA_Known	exon	543097	545706		.	+	.
    ENm006	VEGA_Known	exon	545879	545953		.	+	.
    ENm006	VEGA_Known	exon	546315	546508		.	+	.
    ENm006	VEGA_Known	exon	546980	547020		.	+	.
    ENm006	VEGA_Known	exon	547684	547769		.	+	.
    ENm006	VEGA_Known	exon	548257	548495		.	+	.
    
Dal momento che lo strand dei *record* è `+`, la *feature* `(542747, 542902)` di coordinate minori è il primo esone del trascritto, mentre quella di coordinate maggiori `(548257, 548495)` è l'ultimo esone del trascritto.

**Esempio2**: *features* `exon` che compongono la sequenza del trascritto `U52112.4-018` del gene `ARHGAP4`:

    ENm006	VEGA_Known	exon	79484	79511		.	-	. 
    ENm006	VEGA_Known	exon	72761	72965		.	-	.
    ENm006	VEGA_Known	exon	72521	72683		.	-	.
    ENm006	VEGA_Known	exon	72253	72379		.	-	.
    ENm006	VEGA_Known	exon	71896	71965		.	-	.

Dal momento che lo strand dei *record* è `-`, la *feature* `(79484, 79511)` di coordinate maggiori è il primo esone del trascritto, mentre quella di coordinate minori `(71896, 71965)` è l'ultimo esone del trascritto.

**Esempio3**: *features* `CDS` che compongono la sequenza della *coding sequence* del trascritto `U52112.4-019` del gene `ARHGAP4`:

    ENm006	VEGA_Known	CDS	72761	72963	.	-	0	
    ENm006	VEGA_Known	CDS	72521	72683	.	-	1	
    ENm006	VEGA_Known	CDS	72253	72315	.	-	0	
    ENm006	VEGA_Known	CDS	71872	71965	.	-	0	

Dal momento che lo strand dei *record* è `-`, la *feature* `(72761, 72963)` di coordinate maggiori è la prima *feature* della *coding sequence*, mentre quella di coordinate minori `(71872, 71965)` è l'ultima *feature* della *coding sequence*.

Per ricostruire una sequenza:

1. Ordino la lista di tuple (*features*) per coordinate crescenti
2. Concateno le sequenze delle *feature* secondo l'ordine crescente delle coordinate
3. Se lo strand è `-`, eseguo un *reverse and complement* della sequenza ottenuta

In [8]:
def reconstruct_sequence(feature_list, reference_sequence, strand):
    feature_list.sort()
    
    reconstructed_sequence =''.join(reference_sequence[t[0]-1:t[1]] for t in feature_list)
    
    if strand == '-':
        reconstructed_sequence = reverse_complement(reconstructed_sequence)
        
    return reconstructed_sequence

### 6) Ottenere la genomica di riferimento dal file `FASTA`

In [9]:
with open(reference_file_name, 'r') as input_file:
    reference_file_rows = input_file.readlines()

In [10]:
reference_file_rows

['>ENm006\n',
 '...\n',
 ...]

In [11]:
reference_sequence = ''.join(reference_file_rows[1:]).replace('\n', '')

In [12]:
reference_sequence

'...'

### 7) Selezione dei *record* GTF che servono per ricostruire le sequenze del tipo scelto

In [13]:
with open(gtf_file_name, 'r') as input_file:
    gtf_records = input_file.readlines()

In [14]:
selected_gtf_records = [rec for rec in gtf_records if rec.split('\t')[2] == selected_feature]

In [15]:
for record in selected_gtf_records:
    print(record)

ENm006	VEGA_Known	CDS	71783	71788	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	69989	70210	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	64935	65036	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	64566	64673	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	64385	64459	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	72761	72963	.	-	0	transcript_id "U52112.4-019"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	72521	72683	.	-	1	transcript_id "U52112.4-019"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	72253	72315	.	-	0	transcript_id "U52112.4-019"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	71872	71965	.	-	0	transcript_id "U52112.4-019"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	72761	72963	.	-	0	transcript_id "U52112.4-017"; gene_id "ARHGAP4";

ENm006	VEG

### 8) Costruzione del dizionario degli *strand*

A partire dai *record* selezionati costruire il dizionario:

- *chiave*: identificatore del gene
- *valore*: strand del gene rispetto al *reference*

In [16]:
strand_dict = {}

for gtf_record in selected_gtf_records:
    strand = gtf_record.split('\t')[6]
    
    (gene_id, transcript_id) = get_gene_and_transcript(gtf_record)
    
    strand_dict[gene_id] = strand

In [17]:
strand_dict

{'ARHGAP4': '-', 'AVPR2': '+'}

### 9) Estrazione della lista dei geni per cui è stata annotata almeno una sequenza del tipo scelto

In [18]:
list(strand_dict.keys())

['ARHGAP4', 'AVPR2']

### 10) Ricostruzione delle sequenze del tipo scelto

a) Costruire il dizionario degli identificatori dei trascritti:

   - *chiave*: hugo name del gene
   - *valore*: insieme degli identificatori dei trascritti per cui è annotata la sequenza da ricostruire

In [19]:
gene_dict = {}

for gtf_record in selected_gtf_records:
    (gene_id, transcript_id) = get_gene_and_transcript(gtf_record)
    
    gene_value = gene_dict.get(gene_id, set())
    gene_value.add(transcript_id)
    gene_dict.update([(gene_id, gene_value)])

In [20]:
gene_dict

{'ARHGAP4': {'U52112.4-001',
  'U52112.4-003',
  'U52112.4-005',
  'U52112.4-010',
  'U52112.4-011',
  'U52112.4-017',
  'U52112.4-019',
  'U52112.4-020',
  'U52112.4-024'},
 'AVPR2': {'U52112.2-001', 'U52112.2-003'}}

b) Costruire il dizionario delle *features*:

   - *chiave*: identificatore del trascritto
   - *valore*: lista delle tuple `(start, end, frame)` delle *features* associate al trascritto che compongono la sequenza che si è scelto di ricostruire
    
**NB**: il valore di `frame` sarà *dot* (assenza di informazione) nel caso si scelga di ricostruire i trascritti.

In [21]:
composition_dict = {}

for gtf_record in selected_gtf_records:
    (gene_id, transcript_id) = get_gene_and_transcript(gtf_record)
    
    record_fields = gtf_record.split('\t')
    
    feature_start = int(record_fields[3])
    feature_end = int(record_fields[4])
    frame = record_fields[7]
    
    composition_value = composition_dict.get(transcript_id, list())
    composition_value.append((feature_start, feature_end, frame))
    composition_dict.update([(transcript_id, composition_value)])

In [22]:
composition_dict

{'U52112.4-005': [(71783, 71788, '0'),
  (70312, 70440, '0'),
  (69989, 70210, '0'),
  (64935, 65036, '0'),
  (64566, 64673, '0'),
  (64385, 64459, '0')],
 'U52112.4-019': [(72761, 72963, '0'),
  (72521, 72683, '1'),
  (72253, 72315, '0'),
  (71872, 71965, '0')],
 'U52112.4-017': [(72761, 72963, '0'),
  (72521, 72683, '1'),
  (72253, 72315, '0'),
  (71865, 71965, '0')],
 'U52112.4-010': [(63857, 63942, '2'),
  (62286, 62346, '0'),
  (61857, 61991, '2'),
  (61663, 61768, '2'),
  (61328, 61561, '1'),
  (61169, 61242, '1'),
  (60898, 61081, '2')],
 'U52112.4-020': [(72521, 72560, '1'),
  (71783, 71965, '0'),
  (70724, 70843, '0'),
  (70312, 70440, '0'),
  (70097, 70210, '0')],
 'U52112.4-003': [(77293, 77359, '0'),
  (72761, 72965, '2'),
  (72521, 72683, '1'),
  (72253, 72315, '0'),
  (71783, 71965, '0'),
  (70312, 70440, '0'),
  (69989, 70210, '0'),
  (64935, 65036, '0'),
  (64566, 64757, '0'),
  (64375, 64459, '0'),
  (64181, 64208, '2'),
  (63857, 63959, '1'),
  (62286, 62346, '0'),
  

c) Costruire un dizionario:

   - *chiave*: identificatore di trascritto
   - *valore*: *frame* della prima *feature* che compone la sequenza da ricostruire

In [23]:
frame_dict = {}

for gene_id in gene_dict:
    for transcript_id in gene_dict[gene_id]:
        feature_list = composition_dict[transcript_id]
        sorted_list = sorted(feature_list)
        
        if strand_dict[gene_id] == '+':
            frame = sorted_list[0][2]
        else:
            frame = sorted_list[-1][2]

        frame_dict[transcript_id] = frame

In [24]:
frame_dict

{'U52112.4-005': '0',
 'U52112.4-011': '0',
 'U52112.4-010': '2',
 'U52112.4-020': '1',
 'U52112.4-003': '0',
 'U52112.4-017': '0',
 'U52112.4-001': '0',
 'U52112.4-024': '0',
 'U52112.4-019': '0',
 'U52112.2-003': '0',
 'U52112.2-001': '0'}

d) A partire dai primi due dizionari, ricostruire le sequenze e stamparle in un file `FASTA` con *header* simile a:

    >ARHGAP4; U52112.4-003; len=3235; type=[exon|CDS]; strand=-
    
Se si stanno ricostruendo le *coding sequences*, stampare anche in standard output la separazione in codoni tenendo conto del *frame* della prima *feature*.

In [25]:
output_file_name = './transcripts.fa'
if selected_feature == 'CDS':
    output_file_name = './coding_sequences.fa'
    
with open(output_file_name, 'w') as output_file:
    for gene_id in gene_dict:
        for transcript_id in gene_dict[gene_id]:
            r_sequence = reconstruct_sequence(composition_dict[transcript_id], reference_sequence, strand_dict[gene_id])
            
            header = '>' + gene_id + '; '
            header = header + transcript_id + ' ; len=' + str(len(r_sequence))
            header = header + '; type=' + selected_feature + '; strand ='
            header = header + strand_dict[gene_id]
            
            output_file.write(header + '\n')
            output_file.write(r_sequence + '\n')
            
            if selected_feature == 'CDS':
                frame = int(frame_dict[transcript_id])
                codon_list = re.findall(r'\w{,3}', r_sequence[frame:])
                print(header)
                print('Codone iniziale incompleto: ' + r_sequence[:frame])
                print(' '.join(codon_list))

>ARHGAP4; U52112.4-005 ; len=642; type=CDS; strand =-
Codone iniziale incompleto: 
GAG AAG CGG CAG GCC AAG TTC ATG GAG CAC AAA CTC AAG TGC ACA AAG GCG CGC AAC GAG TAC CTG CTT AGC CTG GCT AGT GTC AAC GCT GCT GTC AGT AAC TAC TAC CTG CAT GAC GTC TTG GAC CTC ATG GAC TGC TGT GAC ACA GGG TTC CAC CTG GCC CTG GGG CAG GTG CTC CGG AGC TAC ACG GCC GCT GAG AGC CGC ACC CAA GCC TCC CAA GTG CAG GGC CTG GGC AGC CTG GAA GAA GCT GTG GAG GCC CTG GAT CCT CCA GGG GAC AAA GCC AAG GTT CTC GAG GTG CAT GCT ACC GTC TTC TGT CCC CCG CTG CGC TTT GAC TAC CAC CCC CAT GAT GGG GAT GAG GTG GCT GAG ATC TGC GTT GAA ATG GAG CTG CGG GAC GAG ATT CTG CCC AGA GCC CAG AAC ATC CAG AGC CGC CTG GAC CGA CAG ACC ATT GAG ACA GAG GAG ACC AGC CCC TCC ACC GAG TCC CTC AAG TCC ACC AGC TCA GAC CCA GGC AGC CGG CAG GCG GGC CGG AGG CGC GGC CAG CAG CAG GAG ACC GAA ACC TTC TAC CTC ACG AAG CTC CAG GAG TAT CTG AGT GGA CGG AGC ATC CTC GCC AAG CTG CAG GCC AAG CAC GAG AAG CTG CAG GAG GCC 
>ARHGAP4; U52112.4-011 ; len=2778; type=CDS; strand =-
Codon